# Subword Tokenization

In this exercise, we will learn how to train our own subword tokenizers with different algorithms: BPE and Unigram. We will use `sentencepiece`, a library from Google to help create our tokenizers.

## Ref:

https://github.com/google/sentencepiece/blob/master/python


## Setup


In [1]:
!wget https://github.com/Knight-H/thai-lm/raw/refs/heads/master/data/pra-apai-manee-ch1-50.txt
!wget https://github.com/Knight-H/thai-lm/raw/refs/heads/master/data/kratoo-40000000-40002000.jsonl

--2025-01-15 13:41:32--  https://github.com/Knight-H/thai-lm/raw/refs/heads/master/data/pra-apai-manee-ch1-50.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Knight-H/thai-lm/refs/heads/master/data/pra-apai-manee-ch1-50.txt [following]
--2025-01-15 13:41:33--  https://raw.githubusercontent.com/Knight-H/thai-lm/refs/heads/master/data/pra-apai-manee-ch1-50.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3231076 (3.1M) [application/octet-stream]
Saving to: ‘pra-apai-manee-ch1-50.txt’

pra-apai-manee-ch1- 100%[===================>]   3.08M  7.11MB/s    in 0.4s    

2025-01-15 13:41:34 (7

## Code


In [12]:
import sentencepiece as spm
import io
import json

Load data


In [13]:
pantip_text = []
with open("kratoo-40000000-40002000.jsonl", "r") as json_file:
    json_list = list(json_file)
    for json_str in json_list:
        result = json.loads(json_str)
        pantip_text.append(f"{result['title']}\n{result['content']}\n")
sum([len(t) for t in pantip_text])

1060318

In [14]:
with open("pra-apai-manee-ch1-50.txt") as f:
    pra_apai_manee_data = f.readlines()
sum([len(t) for t in pra_apai_manee_data])

1100605

In [15]:
pantip_train_text = pantip_text[: int(len(pantip_text) * 0.8)]
pantip_test_text = pantip_text[int(len(pantip_text) * 0.8) :]

pam_train_text = pra_apai_manee_data[
    : int(len(pra_apai_manee_data) * 0.8)
]  # pam = pra_apai_manee
pam_test_text = pra_apai_manee_data[int(len(pra_apai_manee_data) * 0.8) :]

## Run tokenizer training

The Python wrapper provides multiple APIs for training our tokenizers

1. `spm.SentencePieceTrainer.train(input='input.txt', model_prefix='m', vocab_size=vocab_size, model_type=model_type)`
   <br> This will output the tokenizer files `m.model` and `m.vocab` that can be later loaded into `SentencePieceProcessor`.
   <br><br>
2. `spm.SentencePieceTrainer.train(sentence_iterator=iterator, model_writer=obj_with_write_method, vocab_size=vocab_size, model_type=model_type)`
   <br> This method will require a file object e.g. `obj_with_write_method = io.BytesIO()`. The advantage of this method is you can run sentencepiece on environments that have limited access to the local file system. But you will still have to save the model file if you want to re-use the model else you will have to train it again.
   <br><br>
3. `spm.SentencePieceTrainer.train('--input=input.txt --model_prefix=m --vocab_size=vocab_size --model_type=model_type')`
   <br> Same as no.1


### Unigram tokenizer

We are going to start with training a unigram tokenizer. You can use any method of training one. Make sure to set vocab_size to 1000.


In [16]:
train_text = pantip_train_text + pam_train_text
test_text = pantip_test_text + pam_test_text

train_corpus = "\n".join(train_text)
test_corpus = "\n".join(test_text)

with open("train_corpus.txt", "w") as f:
    f.write(train_corpus)

In [34]:
train_corpus_pam = "\n".join(pam_train_text)
test_corpus_pam = "\n".join(pam_test_text)

with open("train_corpus_pam.txt", "w") as f:
    f.write(train_corpus_pam)

In [35]:
## Train
spm.SentencePieceTrainer.train(
    input="train_corpus_pam.txt",
    model_prefix="unigram_pam",
    vocab_size=1000,
    model_type="unigram",
)

### Q1 MCV

How many tokens did you get when tokenizing the following sentence with your unigram tokenizer: <br>
'อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม'


In [54]:
unigram_pam_tokenizer = spm.SentencePieceProcessor(model_file='unigram_pam.model')

In [55]:
len(unigram_pam_tokenizer.encode("อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม", out_type=str))

29

### BPE Tokenizer

Now try training a BPE tokenizer.


In [42]:
## Train
spm.SentencePieceTrainer.train(
    input="train_corpus_pam.txt",
    model_prefix="bpe_pam",
    vocab_size=1000,
    model_type="bpe",
)

### Q2 MCV

How many tokens did you get when tokenizing the following sentence with your BPE tokenizer: <br>
'อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม'


In [56]:
bpe_pam_tokenizer = spm.SentencePieceProcessor(model_file='bpe_pam.model')

In [57]:
len(bpe_pam_tokenizer.encode("อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม", out_type=str))

28

These are some of your vocabs. Note that you will see "▁" (U+2581) in every type of tokenizer in SentencePiece since it makes it possible to perform detokenization \(unsplit your sentences\) without relying on language-specific resources.


In [50]:
unigram_vocabs = [sp_pam.id_to_piece(id) for id in range(sp_pam.get_piece_size())]
" | ".join(unigram_vocabs[:500])

'<unk> | <s> | </s> | ▁ | า | เ | น | ม | ย | ก | ร | ว | ด | ส | ง | บ | ค | มา | อ | ล | จะ | ท | ให้ | ห | ไป | ไม่ | แ | ว่า | พ | ุ | ี | ๏ | ฯ | ข | ช | เป็น | พระ | โ | ที่ | ใจ | ▁จะ | จ | ะ | ิ | ต | ก็ | อยู่ | ป | ได้ | ่ | ไ | เข้า | ู | ▁พระ | ้า | ตาม | ใน | ้ | ▁แล้ว | เหมือน | รา | ศ | เจ้า | เห็น | ลา | กัน | ั | หา | นาง | ทรง | ประ | ์ | ยา | ัก | ํา | ซ | าน | ัง | ฉ | องค์ | ัด | แล้ว | อน | ดู | ถ | ด้วย | มี | ▁จึง | นี้ | ่า | ผ | น้อง | แต่ | ทํา | ▁นาง | ▁ให้ | รัก | พี่ | คิด | ลูก | พา | รู้ | การ | กับ | ัน | หน้า | กระ | วน | ออก | ่อ | เขา | ถึง | ระ | ข้า | ับ | พล | นั่ง | ทั้ง | หน | รับ | ษ | กล | วง | ลง | ฝ | กร | พร | ความ | เสีย | ดี | ขึ้น | อง | ่ง | ธ | ▁แต่ | คน | กลับ | ▁ฝ่าย | ้น | อด | ภ | หรือ | ตร | ือ | ฟัง | แม่ | ▁ไม่ | ไว้ | ยัง | ▁เห็น | นา | ขอ | มิ | น้ํา | หล | ดัง | ▁พอ | ▁ทั้ง | ช่วย | สม | นั้น | ริ | ทัพ | ต้อง | วัน | อา | น้อย | รบ | ิน | อย่า | เอา | จน | เรา | สุด | เสียง | ข้าง | หลัง | ตี | ตัว | ละ | สุ | วัง | ทุก | ่น

In [51]:
bpe_vocabs = [bpe.id_to_piece(id) for id in range(bpe.get_piece_size())]
" | ".join(bpe_vocabs[:500])

'<unk> | <s> | </s> | ้า | ่า | อง | ระ | ํา | รา | อย | ่ง | มา | จะ | ัง | ัน | ▁เ | าย | ้ว | ับ | ี่ | ม่ | อน | ให | าม | ้น | ็น | พระ | ีย | าง | กล | ้ง | ัก | หน | ให้ | ไม่ | หล | ่น | ึง | ▁แ | ทั | ตร | าร | ้อง | ไป | ิด | ข้า | ว่า | หม | คร | ือ | ล้ว | เป | เส | ประ | าน | ั่ง | ▁๏ | ▁ฯ | ที่ | อก | เล | ิน | ได | พล | ทร | ัด | นาง | ึก | ได้ | ู่ | ▁จะ | ค์ | ี้ | พร | เป็น | สุ | ทั้ง | อม | ัย | เร | ห็น | ▁จ | ▁พระ | ก็ | ใจ | อา | ื่ | ่าง | ต่ | กร | ิง | วง | วน | ือน | เจ | ู้ | ียง | อยู่ | รร | ตาม | ▁พ | ้วย | าว | ถึง | คล | ั้น | รี | เข | ด้วย | สม | องค์ | สน | าก | ▁แล้ว | เช | ัว | ย์ | ใน | คว | น้ | หมือน | ▁ส | ูก | อบ | กระ | เจ้า | ทรง | ลา | กัน | มี | ่าย | พรา | ิ่ง | เข้า | เห็น | ิต | สง | อด | ณ์ | วย | ้ม | คิด | เม | เก | เด | ▁นาง | วา | ุก | ▁ให้ | ดู | หา | ▁อ | ▁จึง | ทํา | ลง | รัก | เค | แล้ว | ่าน | พี่ | เหมือน | ั่น | ความ | ยง | อย่า | หร | มิ | ืน | ช่ | การ | ัญ | ▁ไม่ | ฝ่าย | ศรี | ้าง | วก | ้อม | ือง | น้อง | ยว | พา | แก |

### User-defined symbols

Another important concept to know of is User-defined symbols. These special symbols are reserved for a special purpose \(e.g.\, the \<MASK\> token used in BERT) and will always be tokenized into one token.

Refer to the documentation for ways to add these special tokens to your tokenizer.

https://github.com/google/sentencepiece/blob/master/python


## Train another tokenizer on another domain

Now try training another unigram tokenizer on `pantip_text` and we will use it to compare with the unigram tokenizer we trained earlier.


In [52]:
train_corpus_pantip = "\n".join(pantip_train_text)
test_corpus_pantip = "\n".join(pantip_test_text)

with open("train_corpus_pantip.txt", "w") as f:
    f.write(train_corpus_pantip)

In [53]:
## Train
spm.SentencePieceTrainer.train(
    input="train_corpus_pantip.txt",
    model_prefix="unigram_pantip",
    vocab_size=1000,
    model_type="unigram",
)

In [ ]:
bpe_pam = spm.SentencePieceProcessor(model_file='bpe_pam.model')

## Analyse top tokens on different datasets

Use your tokenizers to tokenize the datasets and analyse your most common vocabularies (try 300-400 vocabs with len>1). Hint: tokenize your data and count the tokens.


In [58]:
unigram_pantip_tokenizer = spm.SentencePieceProcessor(model_file='unigram_pantip.model')

In [64]:
from collections import defaultdict

In [ ]:
tokens_pam = defaultdict(lambda: 0.0)
for sent in pam_test_text:
    tokens = unigram_pam_tokenizer.encode(sent, out_type=str)
    for token in tokens:
        tokens_pam[token] += 1

top_tokens_pam = sorted(tokens_pam.items(), key=lambda x: x[1], reverse=True)

tokens_pantip = defaultdict(lambda: 0.0)
for sent in pantip_test_text:
    tokens = unigram_pantip_tokenizer.encode(sent, out_type=str)
    for token in tokens:
        tokens_pantip[token] += 1

top_tokens_pantip = sorted(tokens_pantip.items(), key=lambda x: x[1], reverse=True)

In [67]:
top_tokens_pam 

[('▁', 21314.0),
 ('เ', 6614.0),
 ('า', 5493.0),
 ('น', 5018.0),
 ('ม', 4969.0),
 ('ก', 4802.0),
 ('ย', 4612.0),
 ('ร', 4409.0),
 ('ว', 4163.0),
 ('ส', 3941.0),
 ('บ', 3857.0),
 ('ด', 3820.0),
 ('ง', 3769.0),
 ('ค', 3051.0),
 ('มา', 3002.0),
 ('ท', 2876.0),
 ('อ', 2857.0),
 ('ล', 2780.0),
 ('จะ', 2506.0),
 ('ห', 2411.0),
 ('ไป', 2407.0),
 ('ให้', 2391.0),
 ('แ', 2271.0),
 ('ว่า', 2214.0),
 ('ช', 2098.0),
 ('ุ', 2087.0),
 ('ไม่', 2085.0),
 ('ี', 1996.0),
 ('พ', 1953.0),
 ('ข', 1933.0),
 ('๏', 1923.0),
 ('ฯ', 1922.0),
 ('โ', 1789.0),
 ('ที่', 1632.0),
 ('เป็น', 1587.0),
 ('▁จะ', 1580.0),
 ('พระ', 1561.0),
 ('ต', 1535.0),
 ('ะ', 1512.0),
 ('จ', 1504.0),
 ('ิ', 1461.0),
 ('ใจ', 1458.0),
 ('ก็', 1368.0),
 ('อยู่', 1339.0),
 ('ป', 1329.0),
 ('ั', 1326.0),
 ('์', 1296.0),
 ('ไ', 1280.0),
 ('▁พระ', 1251.0),
 ('ได้', 1247.0),
 ('ศ', 1197.0),
 ('เข้า', 1190.0),
 ('ู', 1152.0),
 ('รา', 1128.0),
 ('ตาม', 1096.0),
 ('้า', 1086.0),
 ('▁แล้ว', 1085.0),
 ('ใน', 1069.0),
 ('ยา', 1063.0),
 ('่', 1061.0)

### To answer

What are some notable differences you see between the two vocabs?

Write your answer below.


## Using tokenizer across domains

One problem you may face is your dataset is very specialized. In that case the tokenizer trained on a general domain may not perform as good as it should when used on your dataset.

Next you will try using tokenizers trained on one general domain (on Pantip) and use it on a specialized domain (พระอภัยมณี) and vice versa.


### Q3 MCV

What percentage increase do you observe when tokenizing the whole พระอภัยมณี dataset with a tokenizer trained on Pantip compared to the one trained on พระอภัยมณี.


### Q4 MCV

What percentage increase do you observe when tokenizing the whole Pantip dataset with a tokenizer trained on พระอภัยมณี compared to the one trained on Pantip.


### To answer

Why do you think the number of tokens tokenized by the general tokenizer (the one trained on Pantip) has a higher percentage increase compared to the number of tokens tokenized by the specialized tokenizer? (Hint: we fixed vocab size.)


## The effect on language models

Next, we will see the effect of using "cross-domain" tokenizers on Language models.


### Setup

We are going to reuse the code from the last assignment


In [ ]:
!pip install lightning

In [ ]:
import itertools
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import lightning as L
from tqdm import tqdm
import numpy as np

In [ ]:
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, seq_len=128):

        token_ids = [tokenizer.encode(d, add_bos=True, add_eos=True) for d in data]
        flatten_token_ids = list(itertools.chain(*token_ids))
        encoded = torch.LongTensor(flatten_token_ids)

        left_over = len(encoded) % seq_len
        encoded = encoded[: len(encoded) - left_over]
        self.encoded = encoded.view(-1, seq_len)

    def __getitem__(self, idx):
        return self.encoded[idx]

    def __len__(self):
        return len(self.encoded)

In [ ]:
class LSTM(L.LightningModule):
    def __init__(
        self,
        vocab_size,
        embedding_dim,
        hidden_dim,
        num_layers,
        dropout_rate,
        learning_rate,
        criterion,
    ):

        super().__init__()

        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=num_layers,
            dropout=dropout_rate,
            batch_first=True,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.learning_rate = learning_rate
        self.criterion = criterion

    def forward(self, src):
        pass

    def training_step(self, batch, batch_idx):

        src = batch[:, :-1]
        target = batch[:, 1:]
        prediction = self(src)
        prediction = prediction.reshape(-1, self.vocab_size)
        target = target.reshape(-1)
        loss = self.criterion(prediction, target)
        self.log("train_loss", loss)
        return loss

    def test_step(self, batch, batch_idx, dataloader_idx=0):

        src = batch[:, :-1]
        target = batch[:, 1:]
        with torch.no_grad():
            prediction = self(src)
        prediction = prediction.reshape(-1, self.vocab_size)
        target = target.reshape(-1)
        loss = self.criterion(prediction, target)
        self.log("test_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)

In [ ]:
vocab_size = sp_pam.get_piece_size()
embedding_dim = 200
hidden_dim = 512
num_layers = 3
dropout_rate = 0.2
lr = 1e-3
criterion = nn.CrossEntropyLoss()
train_batch_size = 64
test_batch_size = 128

### Training


<a name="no1"></a>

#### 1. Training on Pantip data with Pantip tokenizer


In [ ]:
trainer = L.Trainer(max_epochs=10, deterministic=True)
model = LSTM(
    vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, lr, criterion
)

pantip_train_dataset = TextDataset(pantip_train_text, sp_pantip)
pantip_train_loader = DataLoader(
    pantip_train_dataset, batch_size=train_batch_size, shuffle=True
)

pantip_test_dataset = TextDataset(pantip_test_text, sp_pantip)
pantip_test_loader = DataLoader(
    pantip_test_dataset, batch_size=test_batch_size, shuffle=False
)

pam_train_dataset = TextDataset(pam_train_text, sp_pantip)
pam_train_loader = DataLoader(
    pam_train_dataset, batch_size=train_batch_size, shuffle=True
)

pam_test_dataset = TextDataset(pam_test_text, sp_pantip)
pam_test_loader = DataLoader(
    pam_test_dataset, batch_size=test_batch_size, shuffle=False
)

trainer.fit(model, train_dataloaders=pantip_train_loader)

In [ ]:
test_result = trainer.test(
    model,
    dataloaders=[
        pantip_train_loader,
        pam_train_loader,
        pantip_test_loader,
        pam_test_loader,
    ],
    verbose=False,
)

print(
    f"Perplexity on Pantip train set is:\t{np.exp(test_result[0]['test_loss/dataloader_idx_0'])}"
)
print(
    f"Perplexity on Pra apai manee train set is:\t{np.exp(test_result[1]['test_loss/dataloader_idx_1'])}"
)
print(
    f"Perplexity on Pantip test set is:\t{np.exp(test_result[2]['test_loss/dataloader_idx_2'])}"
)
print(
    f"Perplexity on Pra apai manee test set is:\t{np.exp(test_result[3]['test_loss/dataloader_idx_3'])}"
)

<a name="no2"></a>

#### 2. Training on Pantip data with Pra apai manee tokenizer


In [ ]:
trainer = L.Trainer(max_epochs=10, deterministic=True)
model = LSTM(
    vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, lr, criterion
)

pantip_train_dataset = TextDataset(pantip_train_text, sp_pam)
pantip_train_loader = DataLoader(
    pantip_train_dataset, batch_size=train_batch_size, shuffle=True
)

pantip_test_dataset = TextDataset(pantip_test_text, sp_pam)
pantip_test_loader = DataLoader(
    pantip_test_dataset, batch_size=test_batch_size, shuffle=False
)

pam_train_dataset = TextDataset(pam_train_text, sp_pam)
pam_train_loader = DataLoader(
    pam_train_dataset, batch_size=train_batch_size, shuffle=True
)

pam_test_dataset = TextDataset(pam_test_text, sp_pam)
pam_test_loader = DataLoader(
    pam_test_dataset, batch_size=test_batch_size, shuffle=False
)

trainer.fit(model, train_dataloaders=pantip_train_loader)

In [ ]:
test_result = trainer.test(
    model,
    dataloaders=[
        pantip_train_loader,
        pam_train_loader,
        pantip_test_loader,
        pam_test_loader,
    ],
    verbose=False,
)

print(
    f"Perplexity on Pantip train set is:\t{np.exp(test_result[0]['test_loss/dataloader_idx_0'])}"
)
print(
    f"Perplexity on Pra apai manee train set is:\t{np.exp(test_result[1]['test_loss/dataloader_idx_1'])}"
)
print(
    f"Perplexity on Pantip test set is:\t{np.exp(test_result[2]['test_loss/dataloader_idx_2'])}"
)
print(
    f"Perplexity on Pra apai manee test set is:\t{np.exp(test_result[3]['test_loss/dataloader_idx_3'])}"
)

#### To answer

The perplexity numbers should indicate that:

1. Training the LM with Pra apai manee tokenizer on Pantip (no. [2](#no2)) results in overfitting to Pantip and poor generalization to the Pra apai manee dataset.
2. However using the Pantip tokenizer (no. [1](#no1)) results in a much better generalization.

Try and come up with some reasons for the results above. <br>
Hint:

1. think about "general" vocabs and domain-specific vocabs.
2. what do you think happens to the model when the token ids become longer.


<a name="no3"></a>

#### 3. Training on Pra apai manee data with Pantip tokenizer


In [ ]:
trainer = L.Trainer(max_epochs=10, deterministic=True)
model = LSTM(
    vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, lr, criterion
)

pantip_train_dataset = TextDataset(pantip_train_text, sp_pantip)
pantip_train_loader = DataLoader(
    pantip_train_dataset, batch_size=train_batch_size, shuffle=True
)

pantip_test_dataset = TextDataset(pantip_test_text, sp_pantip)
pantip_test_loader = DataLoader(
    pantip_test_dataset, batch_size=test_batch_size, shuffle=False
)

pam_train_dataset = TextDataset(pam_train_text, sp_pantip)
pam_train_loader = DataLoader(
    pam_train_dataset, batch_size=train_batch_size, shuffle=True
)

pam_test_dataset = TextDataset(pam_test_text, sp_pantip)
pam_test_loader = DataLoader(
    pam_test_dataset, batch_size=test_batch_size, shuffle=False
)

trainer.fit(model, train_dataloaders=pam_train_loader)

In [ ]:
test_result = trainer.test(
    model,
    dataloaders=[
        pantip_train_loader,
        pam_train_loader,
        pantip_test_loader,
        pam_test_loader,
    ],
    verbose=False,
)

print(
    f"Perplexity on Pantip train set is:\t{np.exp(test_result[0]['test_loss/dataloader_idx_0'])}"
)
print(
    f"Perplexity on Pra apai manee train set is:\t{np.exp(test_result[1]['test_loss/dataloader_idx_1'])}"
)
print(
    f"Perplexity on Pantip test set is:\t{np.exp(test_result[2]['test_loss/dataloader_idx_2'])}"
)
print(
    f"Perplexity on Pra apai manee test set is:\t{np.exp(test_result[3]['test_loss/dataloader_idx_3'])}"
)

<a name="no4"></a>

#### 4. Training on Pra apai manee data with Pra apai manee tokenizer


In [ ]:
trainer = L.Trainer(max_epochs=10, deterministic=True)
model = LSTM(
    vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, lr, criterion
)

pantip_train_dataset = TextDataset(pantip_train_text, sp_pam)
pantip_train_loader = DataLoader(
    pantip_train_dataset, batch_size=train_batch_size, shuffle=True
)

pantip_test_dataset = TextDataset(pantip_test_text, sp_pam)
pantip_test_loader = DataLoader(
    pantip_test_dataset, batch_size=test_batch_size, shuffle=False
)

pam_train_dataset = TextDataset(pam_train_text, sp_pam)
pam_train_loader = DataLoader(
    pam_train_dataset, batch_size=train_batch_size, shuffle=True
)

pam_test_dataset = TextDataset(pam_test_text, sp_pam)
pam_test_loader = DataLoader(
    pam_test_dataset, batch_size=test_batch_size, shuffle=False
)

trainer.fit(model, train_dataloaders=pam_train_loader)

In [ ]:
test_result = trainer.test(
    model,
    dataloaders=[
        pantip_train_loader,
        pam_train_loader,
        pantip_test_loader,
        pam_test_loader,
    ],
    verbose=False,
)

print(
    f"Perplexity on Pantip train set is:\t{np.exp(test_result[0]['test_loss/dataloader_idx_0'])}"
)
print(
    f"Perplexity on Pra apai manee train set is:\t{np.exp(test_result[1]['test_loss/dataloader_idx_1'])}"
)
print(
    f"Perplexity on Pantip test set is:\t{np.exp(test_result[2]['test_loss/dataloader_idx_2'])}"
)
print(
    f"Perplexity on Pra apai manee test set is:\t{np.exp(test_result[3]['test_loss/dataloader_idx_3'])}"
)

#### To answer

The perplexity numbers should indicate that:

1. Both LM overfits on Pra apai manee data and performs really bad on Pantip data.
2. However using the Pra apai manee tokenizer (no. [4](#no4)) results in a better generalization than the Pantip tokenizer(no. [3](#no3)).

Try and come up with some reasons for the results above. <br>
